### Что изменено?
- Аналогичные лабе 2 изменения (чуть более слабая модель, [немного промпт-инженеринга](https://www.youtube.com/watch?v=Va4_6QtKAwQ))
- Регулируемое число примеров для few-shot промпта
- Для каждого ответа с категорией, которой нет в списке, делаем ещё один запрос модели, чтобы она попробовала уточнить категорию самостоятельно
- **Просить модель сперва обосновать ход "мыслей", а затем дать ответ на основе рассуждений**

In [1]:
!pip install datasets -q
!pip install transformers -q
!pip install accelerate -q
!pip install fuzzywuzzy -q
!pip install python-Levenshtein -q

# https://stackoverflow.com/questions/53247985/tqdm-4-28-1-in-jupyter-notebook-intprogress-not-found-please-update-jupyter-an
!pip install ipywidgets -q
!pip install jupyterlab -q

In [2]:
from typing import Dict, List, Union

import datasets
import transformers

from fuzzywuzzy import fuzz, process

import sklearn.metrics as m
from tqdm.notebook import tqdm
import random

### Функция для создания промптов к модели

**[Здесь и находится ключевое отличие few-shot подхода от zero-shot](https://courses.sberuniversity.ru/llm-gigachat/2/3/2)**

In [3]:
def prepare_message_for_llm(
        text: Union[str, List[str]],
        examples: Dict[str, List[str]],
        prompt_system: str,
        prompt_user: str
    ) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:

    assert len(examples) >= 2, f'Ожидалось 2+ категорий, получено {len(examples)}'

    categories = list(examples.keys())
    categories_as_string = '; '.join(categories)
    # print(examples, categories)

    prompt_user_full = f'{prompt_user}'

    for category in categories:
        if (len(examples[category]) > 0):
            for example in examples[category]:
                prompt_user_full += f'\nПРИМЕР: {example} ' \
                                    f'\nОТВЕТ: {category} '
            # prompt += f'Текст: {" ".join(examples[cur].split())}\nВаш ответ: {cur}\n'

    prompt_user_full += f'\nЗАДАНИЕ: {text} \nСПИСОК ТЕМ (выберите ОДНУ тему СТРОГО ' \
                        f'из этого списка): "{categories_as_string}" \nВАШ ОТВЕТ: '

    messages = [
        {
            'role': 'system',
            'content': prompt_system
        },
        {
            'role': 'user',
            'content': prompt_user_full
        }
    ]

    return {'message_for_llm': messages}

# Pipeline

### Загрузим модель

In [4]:
try:
    print(LLM_PIPELINE)
except:
    LLM_PIPELINE = transformers.pipeline(model='Qwen/Qwen2.5-3B-Instruct', device_map='cuda:0', torch_dtype='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Загрузим датасет

In [5]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = datasets.load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

### Выделим категории

In [6]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Категории классификации текстов: \n{list_of_categories}')

Категории классификации текстов: 
['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [7]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


### Выделим случайные примеры

Тут добавил фичу чтобы менять число примеров, начиная с 3 примеров она или почти не влияет или вообще ухудшает

In [8]:
amount_of_examples = 2
random.seed(2024)

examples_by_categories = dict()

for current_category in list_of_categories:
    category_examples = []

    for ex_no in range(amount_of_examples):
        category_examples.append(
            random.choice(
                train_set.filter(lambda it: it['category'] == current_category)['text']
            )
        )

    examples_by_categories[current_category] = category_examples
    print(f'Категория: "{current_category}" \n{examples_by_categories[current_category]}\n')

Категория: "entertainment" 
['Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься.', 'Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал.']

Категория: "geography" 
['Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года.', 'Некогда древний город Смирна сегодня — современный, развитый и оживленный торговый центр, расположившийся вдоль огромного залива и окруженный горами.']

Категория: "health" 
['Тем, кто тренируется постоянно, требуется больше поддержки по причине негативного отношения к боли и для того, чтобы отличить хронические боли от чувства дискомфорта после обычных физических нагрузок.', 'Они научились мастерски делать ампутации, чтобы спасать пациентов от гангрены, и так же хорошо освоили жгут и артериальные зажимы для приостановки кровотока.']

Категория: "politics" 
[

### Обернём тексты в prompt для llm

In [9]:
# prompt_system_1 = 'Вы — умный помощник, умеющий читать и анализировать тексты ' \
#                   'на русском языке, всегда дающий продуманные верные ответы. '

# prompt_user_1 = 'Прочтите, пожалуйста, следующий набор текстов ниже ' \
#                 'и определите, какая ОДНА тема из списка тем внизу ' \
#                 'НАИБОЛЕЕ представлена. В ответ напишите ОДНУ НАИБОЛЕЕ' \
#                 'подходящую тему из списка, больше ничего. Спасибо! '

In [10]:
prompt_system_1 = 'Вы — очень рассудительный ассистент, умеющий детально ' \
                  'и подробно анализировать тексты на русском языке, ' \
                  'беспрекословно дающий продуманные верные ответы. '

prompt_user_1 = 'Прочтите, пожалуйста, следующий набор текстов ниже ' \
                'и определите, какая ОДНА тема из списка тем внизу ' \
                'НАИБОЛЕЕ представлена. Вы должны выбрать ТОЛЬКО ОДНУ, ' \
                'НАИБОЛЕЕ подходящую тему, взятую СТРОГО ИЗ СПИСКА. ' \
                'В ответ сперва напишите ваш процесс рассуждения, затем ' \
                'ДВА "\n", а затем, ОДНИМ СЛОВОМ, выбранная вами тема. '

In [11]:
validation_set_for_llm = validation_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_1,
        prompt_user_1
    )
)

In [12]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [13]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы — очень рассудительный ассистент, умеющий детально и подробно анализировать тексты на русском языке, беспрекословно дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий набор текстов ниже и определите, какая ОДНА тема из списка тем внизу НАИБОЛЕЕ представлена. Вы должны выбрать ТОЛЬКО ОДНУ, НАИБОЛЕЕ подходящую тему, взятую СТРОГО ИЗ СПИСКА. В ответ сперва напишите ваш процесс рассуждения, затем ДВА "\n", а затем, ОДНИМ СЛОВОМ, выбранная вами тема. \nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда 

### Сгенерируем ответы

In [14]:
print(
    LLM_PIPELINE(validation_set_for_llm['message_for_llm'][60], max_new_tokens=200)[0]['generated_text']
)
print(
    LLM_PIPELINE(validation_set_for_llm['message_for_llm'][60], max_new_tokens=200)[0]['generated_text'][-1]['content'].strip(' \n."').split(' ')[-1].strip(' \n."')
)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'content': 'Вы — очень рассудительный ассистент, умеющий детально и подробно анализировать тексты на русском языке, беспрекословно дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий набор текстов ниже и определите, какая ОДНА тема из списка тем внизу НАИБОЛЕЕ представлена. Вы должны выбрать ТОЛЬКО ОДНУ, НАИБОЛЕЕ подходящую тему, взятую СТРОГО ИЗ СПИСКА. В ответ сперва напишите ваш процесс рассуждения, затем ДВА "\n", а затем, ОДНИМ СЛОВОМ, выбранная вами тема. \nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда 

In [44]:
validation_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=200)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
validation_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

In [87]:
print(validation_pred[1])

[{'content': 'Вы — очень рассудительный ассистент, умеющий детально и подробно анализировать тексты на русском языке, беспрекословно дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий набор текстов ниже и определите, какая ОДНА тема из списка тем внизу НАИБОЛЕЕ представлена. Вы должны выбрать ТОЛЬКО ОДНУ, НАИБОЛЕЕ подходящую тему, взятую СТРОГО ИЗ СПИСКА. В ответ сперва напишите ваш процесс рассуждения, затем ДВА "\n", а затем, ОДНИМ СЛОВОМ, выбранная вами тема. \nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда 

In [66]:
print(m.classification_report(
    y_true=validation_true,
    y_pred=[x[-1]['content'].strip('.,:\'"* \n').split(' ')[-1].strip('.,:\'"* \n') for x in validation_pred])
)

                            precision    recall  f1-score   support

             (путешествие)       0.00      0.00      0.00         0
             entertainment       0.67      0.44      0.53         9
                 geography       0.56      0.62      0.59         8
                    health       0.86      0.55      0.67        11
          history

history       0.00      0.00      0.00         0
                  politics       0.91      0.71      0.80        14
        science/technology       0.81      0.68      0.74        25
                    sports       1.00      0.50      0.67        12
                    travel       0.68      0.65      0.67        20
                         О       0.00      0.00      0.00         0
            будет:

travel       0.00      0.00      0.00         0
выбор:

science/technology       0.00      0.00      0.00         0
                         г       0.00      0.00      0.00         0
                       или       0.00      0.00

### Находим некорректные ответы

In [18]:
validation_clarification_idx = []
validation_clarification_set = []

for i, conversation in enumerate(validation_pred):
    for entry in conversation:
        if entry['role'] == 'assistant' and entry['content'] not in list_of_categories:
            validation_clarification_idx.append(i)

In [19]:
print(f'Ответы с выдуманными категориями ({len(validation_clarification_idx)}):')

for i in validation_clarification_idx:
    validation_clarification_entry = validation_set[i]
    validation_clarification_entry['text'] #+= f' (неверая тема: {validation_pred[i][-1]["content"]})'
    validation_clarification_set.append(validation_clarification_entry)
    
    print(f'№{int(i)} -- {validation_pred[i][-1]["content"]} ({validation_set[i]})')

Ответы с выдуманными категориями (99):
№0 -- Приступим к анализу текста: "Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость." 

Этот текст подчеркивает важность выносливости в условиях усиленной тренировки. Он не прямо говорит о развлечениях, местоположении, здоровье, политике, науке/технологии или спорте. Тема путешествий здесь не обозначена.

Тема, которая наиболее точно соответствует содержанию данного текста - это "health" (здоровье). 

Ваш выбор: health ({'index_id': 548, 'category': 'sports', 'text': 'Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.'})
№1 -- Процесс рассуждения:
В данном тексте говорится о рекламе различных поездок. Название "поездки" указывает на туризм, путешествия. Также в контексте рекламы часто используются термины, связанные с туризмом, 

In [20]:
validation_clarification_set = datasets.Dataset.from_list(validation_clarification_set)

print(validation_clarification_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


### Уточняем некорректные ответы

In [21]:
prompt_system_2 = 'Вы — умный ассистент, умеющий читать и анализировать тексты ' \
                  'на русском языке, всегда дающий продуманные верные ответы. '

prompt_user_2 = 'Робот выполнял разделение текстов на темы, однако ' \
                'не смог определить темы некоторых текстов однозначно. ' \
                'Ваша задача — выбрать одну тему из списка тем ниже. ' \
                'Если ваш ответ будет содержать тему, которой в списке ' \
                'ниже НЕТ, он будет считаться НЕКОРРЕКТНЫМ И ОТКЛОНЁН!'

In [22]:
validation_clarification_set_for_llm = validation_clarification_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_2,
        prompt_user_2
    )
)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [23]:
print(validation_clarification_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [24]:
validation_clarification_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_clarification_set_for_llm['message_for_llm'])
))
validation_clarification_true = validation_clarification_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

In [25]:
print(validation_clarification_pred[0])

[{'content': 'Вы — умный ассистент, умеющий читать и анализировать тексты на русском языке, всегда дающий продуманные верные ответы. ', 'role': 'system'}, {'content': 'Робот выполнял разделение текстов на темы, однако не смог определить темы некоторых текстов однозначно. Ваша задача — выбрать одну тему из списка тем ниже. Если ваш ответ будет содержать тему, которой в списке ниже НЕТ, он будет считаться НЕКОРРЕКТНЫМ И ОТКЛОНЁН!\nПРИМЕР: Это когда люди посещают место, которое очень отличается от их обычной повседневной жизни, чтобы расслабиться и развлечься. \nОТВЕТ: entertainment \nПРИМЕР: Вечер начал певец Санджу Шарма, за ним выступил Джай Шанкар Чаудхари. esented the chhappan bhog bhajan также. Ему аккомпанировал певец Раджу Кханделвал. \nОТВЕТ: entertainment \nПРИМЕР: Пятнадцать из этих метеоритов связывают с метеоритным дождем, прошедшим в июле прошлого года. \nОТВЕТ: geography \nПРИМЕР: Некогда древний город Смирна сегодня — современный, развитый и оживленный торговый центр, расп

### Объединяем результаты

In [26]:
combine_cnt = 0
for i in validation_clarification_idx:
    validation_pred[i] = validation_clarification_pred[combine_cnt]
    combine_cnt += 1

In [27]:
print(m.classification_report(
    y_true=validation_true,
    y_pred=[x[-1]['content'] for x in validation_pred])
)

                                             precision    recall  f1-score   support

                              entertainment       0.57      0.44      0.50         9
fitness 

Ответ "fitness" наиболее подходящ       0.00      0.00      0.00         0
                                  geography       0.62      0.62      0.62         8
                                     health       0.80      0.36      0.50        11
                             nature/animals       0.00      0.00      0.00         0
                                   politics       1.00      0.64      0.78        14
                         science/technology       0.73      0.88      0.80        25
                                   security       0.00      0.00      0.00         0
                                     sports       0.86      0.50      0.63        12
                                     travel       0.53      0.80      0.64        20

                                   accuracy                    

### Постобработка текста

Делаем нечёткое сравнение строк по формуле Левенштейна, выбираем тему из списка с наименьшим расстоянием

In [28]:
validation_pred_norm = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    validation_pred
))

In [29]:
print(m.classification_report(
    y_true=validation_true,
    y_pred=validation_pred_norm)
)

                    precision    recall  f1-score   support

     entertainment       0.50      0.44      0.47         9
         geography       0.62      0.62      0.62         8
            health       0.80      0.36      0.50        11
          politics       1.00      0.64      0.78        14
science/technology       0.71      0.88      0.79        25
            sports       0.75      0.50      0.60        12
            travel       0.53      0.80      0.64        20

          accuracy                           0.67        99
         macro avg       0.70      0.61      0.63        99
      weighted avg       0.70      0.67      0.66        99



# Тестовые данные

In [30]:
test_set_for_llm = test_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_1,
        prompt_user_1
    )
)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [31]:
test_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
test_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
print(m.classification_report(
    y_true=test_true,
    y_pred=[x[-1]['content'].strip('.,:\'"* \n').split(' ')[-1].strip('.,:\'"* \n') for x in test_pred])
)

                    precision    recall  f1-score   support

     entertainment       0.00      0.00      0.00      19.0
         geography       0.00      0.00      0.00      17.0
            health       0.00      0.00      0.00      22.0
          politics       0.00      0.00      0.00      30.0
science/technology       0.00      0.00      0.00      51.0
            sports       0.00      0.00      0.00      25.0
            travel       0.00      0.00      0.00      40.0
                 В       0.00      0.00      0.00       0.0
                 Т       0.00      0.00      0.00       0.0
               виж       0.00      0.00      0.00       0.0
              вижу       0.00      0.00      0.00       0.0
          внимание       0.00      0.00      0.00       0.0
            данном       0.00      0.00      0.00       0.0
               зад       0.00      0.00      0.00       0.0
                оч       0.00      0.00      0.00       0.0
            понима       0.00      0.00

In [33]:
test_clarification_idx = []
test_clarification_set = []

for i, conversation in enumerate(test_pred):
    for entry in conversation:
        if entry['role'] == 'assistant' and entry['content'] not in list_of_categories:
            test_clarification_idx.append(i)

In [34]:
print(f'Ответы с выдуманными категориями ({len(test_clarification_idx)}):')

for i in test_clarification_idx:
    test_clarification_entry = test_set[i]
    test_clarification_entry['text'] #+= f' (неверая тема: {test_pred[i][-1]["content"]})'
    test_clarification_set.append(test_clarification_entry)
    
    print(f'№{int(i)} -- {test_pred[i][-1]["content"]} ({test_set[i]})')

test_clarification_set = datasets.Dataset.from_list(test_clarification_set)

Ответы с выдуманными категориями (204):
№0 -- Процесс рассуждения: 
Т ({'index_id': 1523, 'category': 'science/technology', 'text': 'Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций.'})
№1 -- Процесс рассуждения: В данном ({'index_id': 1644, 'category': 'science/technology', 'text': 'Атомная бомба работает на том принципе, что для того, чтобы много протонов и нейтронов находились в одном ядре, необходима энергия.'})
№2 -- Процесс рассуждения: 
Т ({'index_id': 558, 'category': 'science/technology', 'text': 'Ядро состоит из двух частиц — нейтронов и протонов.'})
№3 -- Процесс рассуждения:
1. ({'index_id': 456, 'category': 'science/technology', 'text': 'Это позволило объективам с переменным фокусным расстоянием создавать снимки качества, сравнимого с качеством, получаемым объективами с постоянным фокусным расстоянием.'})
№4 -- Процесс рассуждения: В данном ({'index_id': 906, 'category': 'science/technology', 'text': 'Они лучше ве

In [35]:
test_clarification_set_for_llm = test_clarification_set.map(
    lambda it: prepare_message_for_llm(
        it['text'],
        examples_by_categories,
        prompt_system_2,
        prompt_user_2
    )
)

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [36]:
test_clarification_pred = list(map(
    lambda x: LLM_PIPELINE(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_clarification_set_for_llm['message_for_llm'])
))
test_clarification_true = test_clarification_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [37]:
combine_cnt = 0
for i in test_clarification_idx:
    test_pred[i] = test_clarification_pred[combine_cnt]
    combine_cnt += 1

In [38]:
print(m.classification_report(
    y_true=test_true,
    y_pred=[x[-1]['content'] for x in test_pred])
)

                                                    precision    recall  f1-score   support

                                           culture       0.00      0.00      0.00         0
                                         education       0.00      0.00      0.00         0
                                     entertainment       0.85      0.58      0.69        19
                                         geography       0.80      0.71      0.75        17
                                            health       0.81      0.77      0.79        22
                                            nature       0.00      0.00      0.00         0
                                          politics       1.00      0.80      0.89        30
                                science/technology       0.90      0.90      0.90        51
                                            sports       1.00      0.68      0.81        25
                                            travel       0.64      0.93      0.

In [39]:
test_pred_norm = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    test_pred
))

In [40]:
print(m.classification_report(
    y_true=test_true,
    y_pred=test_pred_norm)
)

                    precision    recall  f1-score   support

     entertainment       0.79      0.58      0.67        19
         geography       0.80      0.71      0.75        17
            health       0.74      0.77      0.76        22
          politics       1.00      0.80      0.89        30
science/technology       0.90      0.90      0.90        51
            sports       1.00      0.68      0.81        25
            travel       0.62      0.93      0.74        40

          accuracy                           0.80       204
         macro avg       0.83      0.77      0.79       204
      weighted avg       0.84      0.80      0.81       204



Код с этой же моделью, но без изменений логики, даёт f-score = 0.74.

~~Объявляю гойду!~~

In [41]:
UNLOAD_MODEL = False

if UNLOAD_MODEL:
    try:
        del LLM_PIPELINE
    except:
        pass

    from torch.cuda import empty_cache
    empty_cache()
    from gc import collect
    collect()